In [18]:
import pandas as pd
import numpy as np
import os
from functools import reduce
from tabulate import tabulate

In [19]:
total = []
folder = '../Data/Clean'

for dirs in os.listdir(folder):
    if not dirs.endswith(('.txt', 'checkpoints', 'HPS', 'ACS', 'USAFacts', 'graphics', 'ility')):
        #print(dirs)
        for files in os.listdir(os.path.join(folder,dirs)):
            if not files.endswith(('.txt', 'checkpoints', 'STD2019_2020_clean.csv')):
                total.append(pd.read_csv(os.path.join(folder,(os.path.join(dirs,files))), sep = ',', index_col = 0))
                print(os.path.join(folder, os.path.join(dirs, files)))
    
folder2 = '../Data/Intermediate'
for files2 in os.listdir(folder2):
    if not (files2.endswith(('.txt', 'checkpoints', 'hps_processed.csv'))):
            total.append(pd.read_csv(os.path.join(folder2,files2), sep = ',', index_col = 0))
            print(files2)
            print(pd.DataFrame(pd.read_csv(os.path.join(folder2,files2), sep = ',', index_col = 0)).shape)

../Data/Clean/CDC/disagRaceSexHIV2017_2020_clean.csv
../Data/Clean/CDC/CDCSTDagesex20192020_clean.csv
../Data/Clean/CDC/HIVcountyage_clean.csv
../Data/Clean/CDC/HIV_Diagnoses_Prevalence_2008-2021_clean.csv
CDCSTDoverall2000_2020_processed.csv
(67586, 130)
COVIDcounty_cases_processed.csv
(9426, 4)
ACSDP03_merge.csv
(35425, 31)
ACSDP05_merge.csv
(35425, 42)
saipe0010.csv
(32280, 14)


In [20]:
for index, i in enumerate(total):
    if 'year' in i.columns:
        if 'geography' in i.columns:
            i.drop(columns = 'geography', inplace = True)
            print('Done')
        if 'county_name' in i.columns:
            i.drop(columns = 'county_name' , inplace = True)
            print('done')

Done
Done
Done
Done
done


In [21]:
for index, i in enumerate(total):
    print(index)
    i = i[(i['year'] >= 2002) & (i['year'] <= 2020)]
    print(i['year'].unique())
    #for col in i.columns:
        #print(col, ": ", i[col].dtypes)

0
[2017 2018 2019 2020]
1
[2019 2020]
2
[2017 2018 2019 2020]
3
[2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020]
4
[2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015
 2016 2017 2018 2019 2020]
5
[2020]
6
[2016 2010 2017 2011 2014 2019 2012 2015 2020 2018 2013]
7
[2018 2015 2020 2013 2019 2014 2012 2017 2011 2016 2010]
8
[2002 2003 2004 2005 2006 2007 2008 2009]


In [22]:
lst = ['poverty_rate', 'per_capita_income_2005','per_capita_income_2003','per_capita_income_2004','per_capita_income_2002']
for i in lst:
    total[8][i] = pd.to_numeric(total[8][i], errors = 'coerce')

total[8] = total[8].astype('float64')
total[8][['year','fips']] = total[8][['year','fips']].astype('int64')

total[7] = total[7].astype('float64')
total[7][['year','fips']] = total[7][['year','fips']].astype('int64')

total[6] = total[6].astype('float64')
total[6][['year','fips']] = total[6][['year','fips']].astype('int64')

print(8, ':   ', total[8].dtypes)
print(6, ':   ', total[6].dtypes)
print(7, ':   ', total[7].dtypes)
df1 = pd.merge(total[6], total[8], on = ['year', 'fips', 'unemployment_rate','poverty_rate'], how = 'outer')
print(list(df1.columns))
df2 = pd.merge(total[7], total[8], on = ['year', 'fips', 'total_pop'], how = 'outer')
print(list(df2.columns))
total.append(df1)
total.append(df2)
del(total[6])
del(total[8])
del(total[7])

8 :    fips                        int64
year                        int64
employed_persons          float64
unemployment_rate         float64
total_pop                 float64
poverty_rate              float64
per_capita_income_2007    float64
per_capita_income_2006    float64
per_capita_income_2005    float64
per_capita_income_2008    float64
per_capita_income_2003    float64
per_capita_income_2004    float64
per_capita_income_2009    float64
per_capita_income_2002    float64
dtype: object
6 :    per_capita_income_2016                                                                                   float64
employment_pop_ratio                                                                                     float64
unemployment_rate                                                                                        float64
pct_ind_agriculture_forestry_fishing_and_hunting_and_mining                                              float64
pct_ind_construction                        

In [23]:
for index, i in enumerate(total):
    for col in i.columns:
        if i[col].dtypes != 'float64':
            print(col, ": ", i[col].dtypes)

year :  int64
fips :  int64
year :  int64
fips :  int64
year :  int64
fips :  int64
year :  int64
fips :  int64
year :  int64
fips :  int64
year :  int64
fips :  int64
year :  int64
fips :  int64
year :  int64
fips :  int64


In [24]:
cov = df1.columns.intersection(df2.columns)
df3 = pd.merge(df1,df2,on = ['year','fips'], how = 'outer')
print(cov)
#for i in cov:
    #print(i,': ', df[i].dtypes, ' & ', df2[i].dtypes, '\n', df1[df1['year'] >=2010].sort_values(by = ['fips','year'])[i].unique(), 
          #'\n', df2[df2['year'] >=2010].sort_values(by = ['fips','year'])[i].unique())
    
    #print(df1[(df1['fips']==45001) & (df1['year']==2002)].loc[:,i],  "\n", df2[(df2['fips']==45001) & (df2['year']==2002)].loc[:,i],
         #'\n')
    #print(df1[-cov].columns, '\n', df2[-cov].columns)
    #print(df3[i].dtypes)
#df3 = df3[df3['year'] <2010]
print(list(df3.columns))
df3[(df3['fips'] == 45001) & (df3['year']>=2010)].loc[:,'year':].sort_values(by = 'year')

Index(['unemployment_rate', 'poverty_rate', 'year', 'fips', 'employed_persons',
       'total_pop', 'per_capita_income_2007', 'per_capita_income_2006',
       'per_capita_income_2005', 'per_capita_income_2008',
       'per_capita_income_2003', 'per_capita_income_2004',
       'per_capita_income_2009', 'per_capita_income_2002'],
      dtype='object')
['per_capita_income_2016', 'employment_pop_ratio', 'unemployment_rate_x', 'pct_ind_agriculture_forestry_fishing_and_hunting_and_mining', 'pct_ind_construction', 'pct_ind_manufacturing', 'pct_ind_wholesale_trade', 'pct_ind_retail_trade', 'pct_ind_transportation_and_warehousing_and_utilities', 'pct_ind_information', 'pct_ind_finance_and_insurance_and_real_estate_and_rental_and_leasing', 'pct_ind_professional_scientific_and_management_and_administrative_and_waste_management_services', 'pct_ind_educational_services_and_health_care_and_social_assistance', 'pct_ind_arts_entertainment_and_recreation_and_accommodation_and_food_services', 'pct_ind_o

,year,fips,per_capita_income_2010,per_capita_income_2017,per_capita_income_2011,per_capita_income_2014,pct_employment_status!!population_16_years_and_over!!in_labor_force!!civilian_labor_force!!unemployed,per_capita_income_2019,per_capita_income_2012,per_capita_income_2015,...,unemployment_rate_y,poverty_rate_y,per_capita_income_2007_y,per_capita_income_2006_y,per_capita_income_2005_y,per_capita_income_2008_y,per_capita_income_2003_y,per_capita_income_2004_y,per_capita_income_2009_y,per_capita_income_2002_y
5536,2010,45001,16653.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11977,2011,45001,NaN,NaN,17424.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21638,2012,45001,NaN,NaN,NaN,NaN,NaN,NaN,17876.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34520,2013,45001,NaN,NaN,NaN,NaN,7.9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15198,2014,45001,NaN,NaN,NaN,18303.0,6.6,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24859,2015,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18112.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2316,2016,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8757,2017,45001,NaN,19234.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31300,2018,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18418,2019,45001,NaN,NaN,NaN,NaN,NaN,22646.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
cov1 = total[5].columns.intersection(total[4].columns)
df4 = pd.merge(total[5],total[4],on = cov1.to_list(), how = 'outer')
print(cov1)
for i in cov1:
    print(i,': ', total[5][i].dtypes, ' & ', total[4][i].dtypes, '\n', total[5][total[5]['year']>=2010].sort_values(by = ['fips','year'])[i].unique(), 
          '\n', total[4][total[4]['year']>=2010].sort_values(by = ['fips','year'])[i].unique())
    
    #print(df1[(df1['fips']==45001) & (df1['year']==2002)].loc[:,i],  "\n", df2[(df2['fips']==45001) & (df2['year']==2002)].loc[:,i],
         #'\n')
    #print(df1[-cov].columns, '\n', df2[-cov].columns)
    #print(df3[i].dtypes)
#df3 = df3[df3['year'] <2010]
df4[(df4['fips'] == 45001) & (df4['year']>=2010)].loc[:,'year':].sort_values(by = 'year')

Index(['year', 'fips'], dtype='object')
year :  int64  &  int64 
 [2020 2021 2022] 
 [2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020]
fips :  int64  &  int64 
 [ 1001  1003  1005 ... 56041 56043 56045] 
 [ 1001  1003  1005 ... 72149 72151 72153]


,year,fips,covid_cases,cases_chlamydia,cases_early_non-primary_non-secondary_syphilis,cases_gonorrhea,cases_primary_and_secondary_syphilis,rate_per_100000_chlamydia,rate_per_100000_early_non-primary_non-secondary_syphilis,rate_per_100000_gonorrhea,...,rate_per_100000_male_gonorrhea_multiracial,rate_per_100000_male_gonorrhea_native_hawaiian_other_pacific_islander,rate_per_100000_male_gonorrhea_white,rate_per_100000_male_primary_and_secondary_syphilis_american_indian_alaska_native,rate_per_100000_male_primary_and_secondary_syphilis_asian,rate_per_100000_male_primary_and_secondary_syphilis_black_african_american,rate_per_100000_male_primary_and_secondary_syphilis_hispanic_latino,rate_per_100000_male_primary_and_secondary_syphilis_multiracial,rate_per_100000_male_primary_and_secondary_syphilis_native_hawaiian_other_pacific_islander,rate_per_100000_male_primary_and_secondary_syphilis_white
41612,2010,45001,NaN,132.0,0.0,60.0,0.0,521.2,0.0,236.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44830,2011,45001,NaN,174.0,0.0,50.0,0.0,693.8,0.0,199.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48048,2012,45001,NaN,115.0,1.0,26.0,1.0,459.7,4.0,103.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51266,2013,45001,NaN,131.0,1.0,65.0,0.0,526.1,4.0,261.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54484,2014,45001,NaN,117.0,1.0,43.0,0.0,471.9,4.0,173.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57702,2015,45001,NaN,82.0,1.0,36.0,1.0,330.7,4.0,145.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60920,2016,45001,NaN,82.0,0.0,31.0,1.0,332.6,0.0,125.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64138,2017,45001,NaN,160.0,2.0,66.0,1.0,651.3,8.1,268.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67356,2018,45001,NaN,140.0,5.0,55.0,2.0,569.4,20.3,223.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70574,2019,45001,NaN,125.0,1.0,58.0,3.0,509.6,4.1,236.5,...,0.0,0.0,24.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
test = df3
test[test['fips'] == 45001].loc[:,'year':].sort_values(by = 'year')

,year,fips,per_capita_income_2010,per_capita_income_2017,per_capita_income_2011,per_capita_income_2014,pct_employment_status!!population_16_years_and_over!!in_labor_force!!civilian_labor_force!!unemployed,per_capita_income_2019,per_capita_income_2012,per_capita_income_2015,...,unemployment_rate_y,poverty_rate_y,per_capita_income_2007_y,per_capita_income_2006_y,per_capita_income_2005_y,per_capita_income_2008_y,per_capita_income_2003_y,per_capita_income_2004_y,per_capita_income_2009_y,per_capita_income_2002_y
37745,2000,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40969,2001,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44193,2002,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.5,13.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31244.0
47417,2003,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.5,13.6,NaN,NaN,NaN,NaN,30578.0,NaN,NaN,NaN
50641,2004,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.9,15.1,NaN,NaN,NaN,NaN,NaN,31362.0,NaN,NaN
53865,2005,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.4,19.2,NaN,NaN,32486.0,NaN,NaN,NaN,NaN,NaN
57089,2006,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.4,18.1,NaN,34610.0,NaN,NaN,NaN,NaN,NaN,NaN
60313,2007,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.7,17.4,35993.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63537,2008,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.6,17.6,NaN,NaN,NaN,36041.0,NaN,NaN,NaN,NaN
66761,2009,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.4,19.7,NaN,NaN,NaN,NaN,NaN,NaN,33995.0,NaN


In [27]:
df = pd.merge(total[0], total[1], on = ['year', 'fips'], how = 'outer')
j = 2
df[df['fips'] == 45001].loc[:,'year':].sort_values(by = 'year')

,year,fips,cases_female_hiv_diagnoses,cases_female_hiv_prevalence,cases_female_hiv_viral_suppression,cases_female_linkage_to_hiv_care,cases_female_prep_coverage_and_number_of_persons_prescribed,cases_female_receipt_of_hiv_medical_care,cases_male_hiv_diagnoses,cases_male_hiv_prevalence,...,rateper100000_male_primary and secondary syphilis_0_14,rateper100000_male_primary and secondary syphilis_15_19,rateper100000_male_primary and secondary syphilis_20_24,rateper100000_male_primary and secondary syphilis_25_29,rateper100000_male_primary and secondary syphilis_30_34,rateper100000_male_primary and secondary syphilis_35_39,rateper100000_male_primary and secondary syphilis_40_44,rateper100000_male_primary and secondary syphilis_45_54,rateper100000_male_primary and secondary syphilis_55_64,rateper100000_male_primary and secondary syphilis_65+
0,2017,45001,0.0,12.0,NaN,NaN,NaN,NaN,0.0,36.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2459,2018,45001,NaN,11.0,NaN,NaN,NaN,NaN,NaN,36.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4924,2019,45001,0.0,13.0,9.0,0.0,NaN,10.0,0.0,41.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7451,2020,45001,NaN,13.0,6.0,NaN,NaN,7.0,NaN,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


cov = [df.columns.to_list()]
for i in total[1:]:
    cov.append(i.columns.to_list())
df_list = [x for xs in cov for x in xs]
print(df_list)
print([c for c in df.columns if c in df_list])
df[[c for c in df.columns if c in df_list]]

In [28]:
while j < len(total)-3:
    cov = df.columns.intersection(total[j].columns)
    df = pd.merge(df, total[j], on = cov.to_list() , how = 'outer')
    #print((df[df['fips'] == 45001].loc[:,'year':].sort_values(by = 'year').head()))
    #print(j , ':    ', list(df.columns))
    j += 1
    print(df[df['fips'] == 45001].loc[:,'year':].sort_values(by = 'year'))

      year   fips  cases_female_hiv_diagnoses  cases_female_hiv_prevalence  \
0     2017  45001                         0.0                         12.0   
2462  2018  45001                         NaN                         11.0   
4930  2019  45001                         0.0                         13.0   
7463  2020  45001                         NaN                         13.0   

      cases_female_hiv_viral_suppression  cases_female_linkage_to_hiv_care  \
0                                    NaN                               NaN   
2462                                 NaN                               NaN   
4930                                 9.0                               0.0   
7463                                 6.0                               NaN   

      cases_female_prep_coverage_and_number_of_persons_prescribed  \
0                                                   NaN             
2462                                                NaN             
4930       

In [29]:
df5 = pd.merge(df, df4, on = ['year','fips'],how = 'outer')
df5[df5['fips'] == 45001].loc[:,'year':].sort_values(by = 'year')

,year,fips,cases_female_hiv_diagnoses,cases_female_hiv_prevalence,cases_female_hiv_viral_suppression,cases_female_linkage_to_hiv_care,cases_female_prep_coverage_and_number_of_persons_prescribed,cases_female_receipt_of_hiv_medical_care,cases_male_hiv_diagnoses,cases_male_hiv_prevalence,...,rate_per_100000_male_gonorrhea_multiracial_y,rate_per_100000_male_gonorrhea_native_hawaiian_other_pacific_islander_y,rate_per_100000_male_gonorrhea_white_y,rate_per_100000_male_primary_and_secondary_syphilis_american_indian_alaska_native_y,rate_per_100000_male_primary_and_secondary_syphilis_asian_y,rate_per_100000_male_primary_and_secondary_syphilis_black_african_american_y,rate_per_100000_male_primary_and_secondary_syphilis_hispanic_latino_y,rate_per_100000_male_primary_and_secondary_syphilis_multiracial_y,rate_per_100000_male_primary_and_secondary_syphilis_native_hawaiian_other_pacific_islander_y,rate_per_100000_male_primary_and_secondary_syphilis_white_y
43844,2000,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47066,2001,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50288,2002,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53510,2003,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56732,2004,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59954,2005,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63176,2006,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66398,2007,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12617,2008,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15721,2009,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
cov2 = df1.columns.intersection(df5.columns)
df6 = pd.merge(df1,df5,on = cov2.to_list(), how = 'outer')
df6[df6['fips'] == 45001].loc[:,'year':].sort_values(by = 'year')

,year,fips,per_capita_income_2010,per_capita_income_2017,per_capita_income_2011,per_capita_income_2014,pct_employment_status!!population_16_years_and_over!!in_labor_force!!civilian_labor_force!!unemployed,per_capita_income_2019,per_capita_income_2012,per_capita_income_2015,...,rate_per_100000_male_gonorrhea_multiracial_y,rate_per_100000_male_gonorrhea_native_hawaiian_other_pacific_islander_y,rate_per_100000_male_gonorrhea_white_y,rate_per_100000_male_primary_and_secondary_syphilis_american_indian_alaska_native_y,rate_per_100000_male_primary_and_secondary_syphilis_asian_y,rate_per_100000_male_primary_and_secondary_syphilis_black_african_american_y,rate_per_100000_male_primary_and_secondary_syphilis_hispanic_latino_y,rate_per_100000_male_primary_and_secondary_syphilis_multiracial_y,rate_per_100000_male_primary_and_secondary_syphilis_native_hawaiian_other_pacific_islander_y,rate_per_100000_male_primary_and_secondary_syphilis_white_y
38702,2000,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41932,2001,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45162,2002,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48392,2003,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51622,2004,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54852,2005,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58082,2006,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61312,2007,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64546,2008,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67784,2009,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
cov3 = df2.columns.intersection(df5.columns)
df7 = pd.merge(df2,df6,on = cov3.to_list(), how = 'outer')
df7[df7['fips'] == 45001].loc[:,'year':].sort_values(by = 'year')

,year,total_pop_x,fips,pct_sex_age_male,pct_sex_age_female,pct_white,pct_black,pct_native_american,pct_asian,pct_native_hawaiian_pacific_islander,...,rate_per_100000_male_gonorrhea_multiracial_y,rate_per_100000_male_gonorrhea_native_hawaiian_other_pacific_islander_y,rate_per_100000_male_gonorrhea_white_y,rate_per_100000_male_primary_and_secondary_syphilis_american_indian_alaska_native_y,rate_per_100000_male_primary_and_secondary_syphilis_asian_y,rate_per_100000_male_primary_and_secondary_syphilis_black_african_american_y,rate_per_100000_male_primary_and_secondary_syphilis_hispanic_latino_y,rate_per_100000_male_primary_and_secondary_syphilis_multiracial_y,rate_per_100000_male_primary_and_secondary_syphilis_native_hawaiian_other_pacific_islander_y,rate_per_100000_male_primary_and_secondary_syphilis_white_y
38702,2000,26229.0,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41932,2001,26330.0,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45162,2002,26311.0,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48392,2003,26306.0,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51622,2004,26235.0,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54852,2005,25995.0,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58082,2006,25821.0,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61312,2007,25745.0,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64546,2008,25699.0,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67784,2009,25614.0,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
list(df7.columns)

['pct_one_or_more_race',
 'pct_one_or_more_race_white',
 'pct_one_or_more_race_black',
 'pct_one_or_more_race_native_american',
 'pct_one_or_more_race_asian',
 'pct_one_or_more_race_native_hawaiian_pacific_islander',
 'pct_one_or_more_race_other_race',
 'pct_ethnicity_hispanic',
 'pct_sex_age_under_5',
 'pct_sex_age_5_to_9',
 'pct_sex_age_10_to_14',
 'pct_sex_age_15_to_19',
 'pct_sex_age_20_to_24',
 'pct_sex_age_25_to_34',
 'pct_sex_age_35_to_44',
 'pct_sex_age_45_to_54',
 'pct_sex_age_55_to_59',
 'pct_sex_age_60_to_64',
 'pct_sex_age_65_to_74',
 'pct_sex_age_75_to_84',
 'pct_sex_age_85+',
 'median_age',
 'pct_sex_age_16+',
 'pct_sex_age_18+',
 'pct_sex_age_21+',
 'pct_sex_age_62+',
 'pct_sex_age_65+',
 'pct_sex_age_18+_male',
 'pct_sex_age_18+_female',
 'pct_sex_age_65+_male',
 'pct_sex_age_65+_female',
 'year',
 'total_pop_x',
 'fips',
 'pct_sex_age_male',
 'pct_sex_age_female',
 'pct_white',
 'pct_black',
 'pct_native_american',
 'pct_asian',
 'pct_native_hawaiian_pacific_islander',

In [33]:
df = df[(df['year'] >= 2002) & (df['year'] <= 2020)]
df[df['fips'] == 45001].loc[:,'year':].sort_values(by = 'year')

,year,fips,cases_female_hiv_diagnoses,cases_female_hiv_prevalence,cases_female_hiv_viral_suppression,cases_female_linkage_to_hiv_care,cases_female_prep_coverage_and_number_of_persons_prescribed,cases_female_receipt_of_hiv_medical_care,cases_male_hiv_diagnoses,cases_male_hiv_prevalence,...,rate_per_100000_male_gonorrhea_multiracial,rate_per_100000_male_gonorrhea_native_hawaiian_other_pacific_islander,rate_per_100000_male_gonorrhea_white,rate_per_100000_male_primary_and_secondary_syphilis_american_indian_alaska_native,rate_per_100000_male_primary_and_secondary_syphilis_asian,rate_per_100000_male_primary_and_secondary_syphilis_black_african_american,rate_per_100000_male_primary_and_secondary_syphilis_hispanic_latino,rate_per_100000_male_primary_and_secondary_syphilis_multiracial,rate_per_100000_male_primary_and_secondary_syphilis_native_hawaiian_other_pacific_islander,rate_per_100000_male_primary_and_secondary_syphilis_white
49584,2002,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52802,2003,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56020,2004,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59238,2005,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62456,2006,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65674,2007,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11993,2008,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15089,2009,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18195,2010,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21311,2011,45001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


path = '../Data/Clean/ACS'
cov = []

for files in os.listdir(path):
    if not files.startswith('.'):
        cov.append(pd.read_csv(os.path.join(path, files), index_col = 0).columns.to_list())

print(type(cov))
#print(cov)

df_list = [x for xs in cov for x in xs]
print(df_list)
print([c for c in df.columns if c in df_list])
cov = df.columns.intersection(total[j].columns)
df[[c for c in df.columns if c in df_list]]

In [34]:
df.to_csv('../Data/Final/STI0220_cov0220.csv', index = False)